In [1]:
import torch

import os
import sys
sys.path.append(r'../ncg_optimizer')
from fr import FR
#import ncg_optimizer as optim

In [2]:
def quadratic(tensor):
    x, y = tensor
    a = 2.0
    b = 1.0
    return (x ** 2) / a + (y ** 2) / b + 2 * x * y - x
min_loc = (-1, 1)

In [2]:
def rosenbrock(tensor):
    x, y = tensor
    return (1 - x) ** 2 + 1 * (y - x ** 2) ** 2
min_loc = (1, 1)

In [2]:
def beale(tensor):
    x, y = tensor
    f = (
        (1.5 - x + x * y) ** 2
        + (2.25 - x + x * y ** 2) ** 2
        + (2.625 - x + x * y ** 3) ** 2
    )
    return f
min_loc = (3, 0.5)

In [4]:
initial_state = (1.5, 1.5)
MyDevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x = torch.tensor(initial_state,device=MyDevice).requires_grad_(True)
x_min = torch.tensor(min_loc,device=MyDevice)
iterations = 500

In [5]:
#optimizer = FR([x], eps=1e-3, line_search='Armijo', lr=0.1)
#optimizer = FR([x], eps=1e-3, line_search='Wolfe', c2=0.9, lr=0.5, eta=5)
optimizer = FR([x], eps=1e-3, line_search='Strong_Wolfe', c2=0.4, lr=0.2, amax=0.6)
#optimizer = FR([x], eps=1e-3, line_search='None')
for _ in range(iterations):
    def closure():
        print(x)
        optimizer.zero_grad()
        f = quadratic(x)
        #f = rosenbrock(x)
        #f = beale(x)
        f.backward(retain_graph=True, create_graph=True)
        return f
    optimizer.step(closure)
torch.allclose(x, x_min.float(), atol=0.01)

tensor([1.5000, 1.5000], device='cuda:0', requires_grad=True)
tensor([1.5000, 1.5000], device='cuda:0', requires_grad=True)
tensor([1.1500, 0.9000], device='cuda:0', requires_grad=True)
tensor([ 0.4500, -0.3000], device='cuda:0', requires_grad=True)
tensor([ 0.4500, -0.3000], device='cuda:0', requires_grad=True)
tensor([ 0.4500, -0.3000], device='cuda:0', requires_grad=True)
tensor([1.5000, 1.5000], device='cuda:0', requires_grad=True)


UnboundLocalError: local variable 'alpha' referenced before assignment